# 接續image_process_view_csv.ipynb

# 下載資料

In [1]:
# 參考網站
# https://towardsdatascience.com/master-the-coco-dataset-for-semantic-image-segmentation-part-1-of-2-732712631047
# https://cocodataset.org/

In [2]:
import fiftyone

In [4]:
dataset = fiftyone.zoo.load_zoo_dataset(
    "coco-2017",
    split="train",
    label_types=["segmentations"],
    classes=["person"]
)

Found annotations at 'C:\Users\sky66\fiftyone\coco-2017\raw\instances_train2017.json'
  22% |███|----------| 13891/64115 [26.2m elapsed, 1.6h remaining, 9.0 images/s]  


KeyboardInterrupt: 

In [ ]:
dataset = fiftyone.zoo.load_zoo_dataset(
    "coco-2017",
    split="validation",
    label_types=["segmentations"],
    classes=["person"]
)

# 調整圖片大小

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
# 放舊圖片的資料夾
old = r"C:\Users\sky66\fiftyone\coco-2017\validation\data"
old1 = r"C:\Users\sky66\fiftyone\coco-2017\train\data"
# 放新圖片的新資料夾
images = r"C:\Users\sky66\fiftyone\coco-2017\new"
size = (256,256)
for e in os.listdir(old):
    fil = old+"\\"+e
    fil1 = images+"\\val_"+e
    a = cv2.imread(fil)
    a = cv2.resize(a, size)
    cv2.imwrite(fil1, a)
for e in os.listdir(old1):
    fil = old1+"\\"+e
    fil1 = images+"\\train_"+e
    a = cv2.imread(fil)
    a = cv2.resize(a, size)
    cv2.imwrite(fil1, a)    

In [ ]:
#顯示檔案的前幾行內容
def view(nam, row):
    with open(nam,"rb") as infile:  
        c = (infile.read(5024))
        [print(i) for i in c.split(b"{")]
    return c  

In [ ]:
view(r"C:\Users\sky66\fiftyone\coco-2017\raw\captions_train2017.json", 1)

# 模組訓練(這裡程式碼沒調好，不用運行)

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.losses import binary_crossentropy
import matplotlib.pyplot as plt
import os
import numpy as np
import cv2

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
# 參考網站
# https://www.kaggle.com/code/phylake1337/99-loss-simple-sol-using-u-net-keras-tf
# https://github.com/petrosgk/Kaggle-Carvana-Image-Masking-Challenge/blob/master/model/u_net.py
# https://www.gushiciku.cn/pl/pdkd/zh-tw
# https://www.tensorflow.org/tutorials/keras/save_and_load

In [ ]:
def cv2plt(img_np):
    img_np = cv2.cvtColor(img_np, cv2.COLOR_BGR2RGB)
    img_np = img_np/255
    return img_np

def plt2cv(img_np):
    return cv2.cvtColor((img_np*255).astype("uint8"), cv2.COLOR_RGB2BGR)

In [ ]:
def load_img(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    return img

def load_mask(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    shp = img.shape
    img = np.reshape(img, (shp[0], shp[1], 1))
    return img

In [ ]:
a = load_mask("C:\\Users\\sky66\\fiftyone\\open-images-v6\\validation\\labels\\new\\c9c492853eb2af36.png")
b = load_img("C:\\Users\\sky66\\fiftyone\\open-images-v6\\validation\\labels\\new\\c9c492853eb2af36.jpg")

In [ ]:
plt.imshow(b)

In [ ]:
plt.imshow(a, cmap="gray")

In [ ]:
path = "C:\\Users\\sky66\\fiftyone\\open-images-v6\\validation\\labels\\new"

# 放mask
newd = []

# 放新圖片
images = []

for i in os.listdir(path):
    if i.split(".")[1]=="png":
        newd.append(i)
    else:
        images.append(i)
        
mask = sorted(newd)
image = sorted(images)

# 取訓練用的80%當驗證用圖片
train = int(len(mask)*0.8)
val = len(image)-train

train_img = image[0:train]
train_mask = mask[0:train]

val_img = image[train:]
val_mask = mask[train:]

input_size = [256, 256, 3]

def data_generator(images_path, masks_path, image_ids, mask_ids, batch_size, img_size=input_size):
    '''
    images_path/masks_path: Images/Masks folder directory.
    images_ids/mask_ids: Ids for '.jpg' images/masks.
    img_size: Generated imgs/masks size.
    
    returns: batch of randomly-selected car&mask images value-scaled (0 -> 1). 
    '''
    data_size = len(image_ids)
    while True:
        #Choose random indice for later picking.
        rnd_ind = np.random.choice(np.arange(data_size),batch_size)
        imgs = []
        masks = []
        for i in rnd_ind:
            #Pick a random id for car&mask images.
            img_id, mask_id = image_ids[i], mask_ids[i]
            #Load/resize images.
            img = load_img(images_path +"\\" + img_id) 
            mask = load_mask(masks_path +"\\" + mask_id)
            #Add to the batch data.
            imgs.append(img)
            masks.append(mask)
        yield np.array(imgs, dtype=np.float16) / 255., np.array(masks, dtype=np.float16) / 255.
        
def dice_coef(y_true, y_pred):
    '''
    Metric
    '''
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score


def dice_loss(y_true, y_pred):
    '''
    Loss function
    '''
    loss = 1 - dice_coef(y_true, y_pred)
    return loss


def bce_dice_loss(y_true, y_pred):
    '''
    Mixed crossentropy and dice loss.
    '''
    loss = binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss     

def get_unet_256(input_shape=(256, 256, 3),
                 num_classes=1):
    inputs = Input(shape=input_shape)
    # 256

    down0 = Conv2D(32, (3, 3), padding='same')(inputs)
    down0 = BatchNormalization()(down0)
    down0 = Activation('relu')(down0)
    down0 = Conv2D(32, (3, 3), padding='same')(down0)
    down0 = BatchNormalization()(down0)
    down0 = Activation('relu')(down0)
    down0_pool = MaxPooling2D((2, 2), strides=(2, 2))(down0)
    # 128

    down1 = Conv2D(64, (3, 3), padding='same')(down0_pool)
    down1 = BatchNormalization()(down1)
    down1 = Activation('relu')(down1)
    down1 = Conv2D(64, (3, 3), padding='same')(down1)
    down1 = BatchNormalization()(down1)
    down1 = Activation('relu')(down1)
    down1_pool = MaxPooling2D((2, 2), strides=(2, 2))(down1)
    # 64

    down2 = Conv2D(128, (3, 3), padding='same')(down1_pool)
    down2 = BatchNormalization()(down2)
    down2 = Activation('relu')(down2)
    down2 = Conv2D(128, (3, 3), padding='same')(down2)
    down2 = BatchNormalization()(down2)
    down2 = Activation('relu')(down2)
    down2_pool = MaxPooling2D((2, 2), strides=(2, 2))(down2)
    # 32

    down3 = Conv2D(256, (3, 3), padding='same')(down2_pool)
    down3 = BatchNormalization()(down3)
    down3 = Activation('relu')(down3)
    down3 = Conv2D(256, (3, 3), padding='same')(down3)
    down3 = BatchNormalization()(down3)
    down3 = Activation('relu')(down3)
    down3_pool = MaxPooling2D((2, 2), strides=(2, 2))(down3)
    # 16

    down4 = Conv2D(512, (3, 3), padding='same')(down3_pool)
    down4 = BatchNormalization()(down4)
    down4 = Activation('relu')(down4)
    down4 = Conv2D(512, (3, 3), padding='same')(down4)
    down4 = BatchNormalization()(down4)
    down4 = Activation('relu')(down4)
    down4_pool = MaxPooling2D((2, 2), strides=(2, 2))(down4)
    # 8

    center = Conv2D(1024, (3, 3), padding='same')(down4_pool)
    center = BatchNormalization()(center)
    center = Activation('relu')(center)
    center = Conv2D(1024, (3, 3), padding='same')(center)
    center = BatchNormalization()(center)
    center = Activation('relu')(center)
    # center

    up4 = UpSampling2D((2, 2))(center)
    up4 = concatenate([down4, up4], axis=3)
    up4 = Conv2D(512, (3, 3), padding='same')(up4)
    up4 = BatchNormalization()(up4)
    up4 = Activation('relu')(up4)
    up4 = Conv2D(512, (3, 3), padding='same')(up4)
    up4 = BatchNormalization()(up4)
    up4 = Activation('relu')(up4)
    up4 = Conv2D(512, (3, 3), padding='same')(up4)
    up4 = BatchNormalization()(up4)
    up4 = Activation('relu')(up4)
    # 16

    up3 = UpSampling2D((2, 2))(up4)
    up3 = concatenate([down3, up3], axis=3)
    up3 = Conv2D(256, (3, 3), padding='same')(up3)
    up3 = BatchNormalization()(up3)
    up3 = Activation('relu')(up3)
    up3 = Conv2D(256, (3, 3), padding='same')(up3)
    up3 = BatchNormalization()(up3)
    up3 = Activation('relu')(up3)
    up3 = Conv2D(256, (3, 3), padding='same')(up3)
    up3 = BatchNormalization()(up3)
    up3 = Activation('relu')(up3)
    # 32

    up2 = UpSampling2D((2, 2))(up3)
    up2 = concatenate([down2, up2], axis=3)
    up2 = Conv2D(128, (3, 3), padding='same')(up2)
    up2 = BatchNormalization()(up2)
    up2 = Activation('relu')(up2)
    up2 = Conv2D(128, (3, 3), padding='same')(up2)
    up2 = BatchNormalization()(up2)
    up2 = Activation('relu')(up2)
    up2 = Conv2D(128, (3, 3), padding='same')(up2)
    up2 = BatchNormalization()(up2)
    up2 = Activation('relu')(up2)
    # 64

    up1 = UpSampling2D((2, 2))(up2)
    up1 = concatenate([down1, up1], axis=3)
    up1 = Conv2D(64, (3, 3), padding='same')(up1)
    up1 = BatchNormalization()(up1)
    up1 = Activation('relu')(up1)
    up1 = Conv2D(64, (3, 3), padding='same')(up1)
    up1 = BatchNormalization()(up1)
    up1 = Activation('relu')(up1)
    up1 = Conv2D(64, (3, 3), padding='same')(up1)
    up1 = BatchNormalization()(up1)
    up1 = Activation('relu')(up1)
    # 128

    up0 = UpSampling2D((2, 2))(up1)
    up0 = concatenate([down0, up0], axis=3)
    up0 = Conv2D(32, (3, 3), padding='same')(up0)
    up0 = BatchNormalization()(up0)
    up0 = Activation('relu')(up0)
    up0 = Conv2D(32, (3, 3), padding='same')(up0)
    up0 = BatchNormalization()(up0)
    up0 = Activation('relu')(up0)
    up0 = Conv2D(32, (3, 3), padding='same')(up0)
    up0 = BatchNormalization()(up0)
    up0 = Activation('relu')(up0)
    # 256

    classify = Conv2D(num_classes, (1, 1), activation='sigmoid')(up0)

    model = Model(inputs=inputs, outputs=classify)

    model.compile(optimizer=RMSprop(lr=0.0001), loss=bce_dice_loss, metrics=[dice_coef])

    return model

In [ ]:
# 目標模組
uNet = get_unet_256()

#Prepare callbacks
LR_callback = ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=10, factor=.2, min_lr=.00001)
EarlyStop_callback = EarlyStopping(monitor='val_loss',patience=10, restore_best_weights=True)

#Perpare data generators.
batch_size = 5
train_gen = data_generator(path, path, train_img, train_mask, batch_size=batch_size)
val_gen = data_generator(path, path, val_img, val_mask, batch_size=batch_size)

In [ ]:
# 開始訓練
history = uNet.fit_generator(train_gen, steps_per_epoch=int(train/batch_size),
                             epochs=35, validation_data=val_gen,
                             validation_steps=int(val/batch_size),
                             callbacks=[LR_callback, EarlyStop_callback])